In [1]:
# Create the Solver object for either NEOS or a local solver
def SetUpSolver(Model):
    Model.Options = None
    if Neos:
        Solver = pyo.SolverManagerFactory('neos')   # Solver on NEOS
        if pyo.value(Model.Engine) == 'cplex':   # Linear
            Model.Options = {'timelimit': Model.TimeLimit, 'mipgap': MipGap}
        elif pyo.value(Model.Engine) == 'octeract':   # Linear or non-linear
            Model.Options = {'MAX_SOLVER_TIME': Model.TimeLimit, 'MILP_SOLVER': 'HIGHS'}
        elif pyo.value(Model.Engine) == 'couenne':   # Non-linear
            print('No options for Couenne')
        else:
            print('Unknown NEOS solver when setting options')
    else:
        Solver = pyo.SolverFactory(pyo.value(Model.Engine))   # Local solver installed
        if pyo.value(Model.Engine) == 'couenne':   # Non-linear
            print('No options for Couenne') # Couenne doesn't accept command line options, use couenne.opt instead
        elif pyo.value(Model.Engine) == 'cbc':   # Linear
            Solver.options['seconds'] = pyo.value(Model.TimeLimit)
            Solver.options['log'] = 1   # Default is 1. 2 or 3 provides more detail. Also have slog, which provides much more detail
        elif pyo.value(Model.Engine) == 'appsi_highs':   # Linear
            Solver.options['time_limit'] = pyo.value(Model.TimeLimit)
            Solver.options['log_file'] = 'highs.log'   # Sometimes HiGHS doesn't update the console as it solves, so write log file too
            Solver.options['mip_rel_gap'] = MipGap   # Relative gap. 10 = stop at first feasible solution. 0 = find optimum. 0.1 = within 10% of optimum
            #Solver.options['parallel'] = 'on'
            #Solver.options['mip_heuristic_effort'] = 0.2   # default = 0.05, range = 0..1
        elif pyo.value(Model.Engine) == 'gurobi_direct':
            Solver.options['timelimit'] = pyo.value(Model.TimeLimit)
            Solver.options['logfile'] = 'gurobi.log'
            Solver.options['mipgap'] = MipGap
            Solver.options['solutionlimit'] = SolutionLimit
            #Solver.options['heuristics'] = 0.25   # 0..1 default = 0.05
        else:
            print('Unknown local solver when setting options')
    
    return Solver, Model

In [2]:
# Call either NEOS or a local solver
def CallSolver(Solver, Model):
    if Neos:
        if Model.Options == None:
            Results = Solver.solve(Model, load_solutions = LoadSolution, tee = Verbose, solver = Model.Engine)
        else:
            Results = Solver.solve(Model, load_solutions = LoadSolution, tee = Verbose, solver = Model.Engine, options = Model.Options)
    else:
        Results = Solver.solve(Model, load_solutions = LoadSolution, tee = Verbose)
    
    return Results, Model